In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from data_reader import *
from tweet_to_vec import TweetToVec
from utils import str_to_vector
from abstract_classifier import AbstractClassifier

In [3]:
dr = DataReader(tokenize_with_nlkt)
train_1 = dr.read_data('data/train_text_1.txt')
tags_train_1 = dr.read_tags('data/train_tags_1.txt')
train_2 = dr.read_data('data/train_text_2.txt')
tags_train_2 = dr.read_tags('data/train_tags_2.txt')

test_1 = dr.read_data('data/test_text_1.txt')
tags_test_1 = dr.read_tags('data/test_tags_1.txt')
test_2 = dr.read_data('data/test_text_2.txt')
tags_test_2 = dr.read_tags('data/test_tags_2.txt')

100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 3352760.99it/s]


In [4]:
t2v = TweetToVec('fixed_length', L=8)
t2v.read_embeddings_from_file('embeddings/emb1.txt')

100%|██████████████████████████████| 2123132/2123132 [00:36<00:00, 58314.01it/s]


In [5]:
stinky_tweet = ['@', 'anonymized_account', '@', 'anonymized_account', 'musi', '.', 'innej', 'drogi', 'nie', 'mamy', '.']
vec = t2v.translate_to_vec(stinky_tweet)
vec

! (8, 100)
! 800


array([ 8.029650e-01,  6.937690e-01,  2.279270e-01,  8.501816e+00,
        1.089828e+00,  1.504081e+00, -3.317430e-01,  1.301918e+00,
       -3.096261e+00, -2.016507e+00,  1.093088e+00,  4.620863e+00,
        1.418485e+00, -7.144394e+00, -6.684737e+00, -2.776209e+00,
        7.260940e-01,  3.647446e+00, -2.327481e+00,  1.736108e+00,
        9.418700e-02,  9.609600e-01, -2.452398e+00,  2.564749e+00,
        2.578323e+00, -3.924780e+00, -1.651585e+00,  3.716427e+00,
       -2.265556e+00,  5.984542e+00,  1.563840e+00, -9.595690e-01,
        1.323772e+00,  8.485210e-01,  6.461382e+00,  2.818434e+00,
       -1.387306e+00, -5.613760e-01, -7.480119e+00, -5.307506e+00,
       -4.532910e-01, -7.557918e+00, -2.281390e-01,  1.689189e+00,
        2.749791e+00, -1.653357e+00, -3.522076e+00,  3.145617e+00,
        2.574080e-01, -2.311172e+00,  7.467520e-01,  4.177340e-01,
       -4.543410e-01,  3.426648e+00, -2.498178e+00,  5.843160e+00,
       -2.511564e+00,  9.797800e-01, -6.894606e+00, -1.819210e

In [6]:
vectors = []
for tweet in train_1:
    vec = t2v.translate_to_vec(tweet)
    vectors.append(vec)

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 10

! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 10

! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 10

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 8

! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 100)
! 800
! (8, 10

In [79]:
def group_into_batches(vecs, batch_size = 32):
    
    batches = []
    current_batch = []
    
    for v in vecs:
        current_batch.append(v)
        #print(v.shape)
        if len(current_batch) == batch_size:
            batches.append(np.array(current_batch))
            current_batch = []
    
    if len(current_batch):
        batches.append(np.array(current_batch))
    
    return batches

In [80]:
def to_tensor(data):
    #TODO: watch out for cpu/gpu!
    return torch.from_numpy(data).float()

In [86]:
batched_vectors = group_into_batches(vectors)
#for batch in batched_vectors:
#    print(batch.shape)
batched_tags = group_into_batches(tags_train_1)
train_data = [to_tensor(batch) for batch in batched_vectors]
train_data_tags = [to_tensor(batch) for batch in batched_tags]

In [87]:
#TODO: init weights
class SimpleNetwork(nn.Module):
    
    def __init__(self, model, loss):
        
        super().__init__()
        self.model = model
        self.loss = loss
        self.init_params()
        
    def init_params(self):
        
        with torch.no_grad():
            for name, p in self.model.named_parameters():
                if "weight" in name:
                    p.normal_(0, np.sqrt(1 / (2 * p.size(dim = 1))))
                elif "bias" in name:
                    p.zero_()
    
    def forward(self, x):
        
        return self.model(x)

    def train(self, training_data, training_classes, epochs, optimizer):
        
        
        optimizer = optimizer(self.model.parameters(), lr = 0.1)
        
        for epoch in range(epochs):
            for (input_batch, true_classes) in zip(training_data, training_classes):
                #tensor_batch = to_tensor(input_batch)
                preds = self.model(input_batch)
                loss = self.loss(preds, true_classes)
                print(loss)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                print(f'After epoch {epoch} loss = {loss}')
        
        

In [88]:
simple_model = nn.Sequential(
                nn.Linear(800, 2))

adam_opt = optim.Adam
ce_loss = torch.nn.CrossEntropyLoss()

In [90]:
train_data[0].dtype, train_data_tags[0].dtype

(torch.float32, torch.float32)

In [91]:
simple_network = SimpleNetwork(simple_model, ce_loss)

simple_network.train(train_data, train_data_tags, 10, adam_opt)

RuntimeError: expected scalar type Long but found Float